# Functions

In [19]:
def get_data(url,arch,username,password):
    '''Download and save data (.txt files)
    url: web link
    arch:local drive path
    username, password: user verification
    '''
    
    try:
        r = requests.get(url,verify=False, auth=(username,password))
        # create beautiful-soup object
        soup = BeautifulSoup(r.content,'html5lib')
        tm = soup.findAll('a')
        links = [url + link['href'] for link in tm if link['href'].endswith('.txt')]
        for link in links:
            # obtain filename by splitting url and getting 
            # last string
            file_name = link.split('/')[-1]           
            # create response object
            r = requests.get(link, verify=False, auth=(username,password))    
            #save file
            open(arch+file_name, 'wb').write(r.content)
            print ("%s downloaded!\n"%file_name)
    except requests.exceptions.RequestException as e: 
        print(e)
        sys.exit(1)
        
    return   

def read_header(fname):
    ''' this reads headers from .txt files with format:
    CTD cast

    DATE:  28-Mar-2014 02:00:00
    latitude:  -63.4998
    longitude:  -175.9997
     P (db)   z (m)    T (^oC)  S (psu)
    0.0000 0.0000 2.0436 33.7868
    2.0000 1.9806 2.0340 33.7873
    4.0000 3.9612 2.0245 33.7878
    6.0000 5.9417 2.0258 33.7869
    ...................................
    '''
    
    f = open(fname, 'r')
    head=f.readlines()[:5]
    date=head[2].split()[1]
    time=head[2].split()[2]
    lat=head[3].split()[1]
    lon=head[4].split()[1]
    f.close()
    return lon,lat,date,time

def read_data(fname):
    '''this reads data from .txt files with format:
    CTD cast

    DATE:  28-Mar-2014 02:00:00
    latitude:  -63.4998
    longitude:  -175.9997
     P (db)   z (m)    T (^oC)  S (psu)
    0.0000 0.0000 2.0436 33.7868
    2.0000 1.9806 2.0340 33.7873
    4.0000 3.9612 2.0245 33.7878
    6.0000 5.9417 2.0258 33.7869
    ...................................
    '''
    
    f = open(fname, 'r')
    #ignore header
    for i in range(5):
             f.readline()
    col_names=f.readline().split()
    col_names=[col_names[0],col_names[2],col_names[4],col_names[6]]
    data_block=f.readlines()
    f.close()
    #create dictionary containing a list of values for each variable
    data={}
    
    for col_name in col_names:
        data[col_name]=MA.zeros(len(data_block),'f',fill_value=-999.999)
             
    for (line_count, line) in enumerate(data_block):
        items = line.split()
        for (col_count, col_name) in enumerate(col_names):
            value = items[col_count]
            data[col_name][line_count] = value
             
    return data

def make_map(lons,lats,labels):
    '''this function creates global map containing station locations
     lon,lats,labels: longitudes,latitudes,names of CTD stations '''
    
    m = Basemap(projection='mill',llcrnrlat=-90,urcrnrlat=90,llcrnrlon=-180,urcrnrlon=180,resolution='c')
    m.drawcoastlines()
    m.fillcontinents(color='tan',lake_color='lightblue')
    # draw parallels and meridians
    m.drawparallels(np.arange(-90.,91.,30.),labels=[True,False,False,False],dashes=[2,2]) #disable it to create transect area subplot
    m.drawmeridians(np.arange(-180.,181.,60.),labels=[False,False,False,True],dashes=[2,2]) #disable it to create transect area subplot
    m.drawmapboundary(fill_color='lightblue')
    # Overlay station locations
    x,y = m(lons, lats)
    m.plot(x, y, 'ro',markersize=8) #marksize=1.5 for transect area plot
    for label, x, y in zip(labels, x,y):
        plt.annotate(
            label,
            xy=(x, y), xytext=(5, +13),
            textcoords='offset points', ha='right', va='bottom',
            #bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
            arrowprops=dict(arrowstyle = '-', connectionstyle='arc3,rad=0'))
    return 
    
def ctd_profile(lon,lat,label,date,time,P,T,S):
    '''this function calculates potential temperature (theta), density (rho) and Brunt-Väisälä frequency squared (N^2) 
    from CSIRO seawater toolbox (https://pypi.python.org/pypi/seawater) and makes station profiles for in-situ temperature,
    potential temperature,salinity,density, Brunt-Väisälä frequency squared.
    
    lon,lat,label:longitude,latitude,name of CTD stations
    date,time: date of sampling
    P,T,S: CTD profiles (Pressure, Temperature, Salinity).
     '''
    
    #calculate Potential Temperature
    theta=sw.ptmp(S, T, P, pr=0)
    
    #calculate density (sigma)
    rho=sw.dens(S, T, P)-1000  #in -situ
    rho_theta=sw.pden(S,T,P,pr=0)-1000  #potential
    
    #calculate Brunt-Väisälä 
    n2=sw.bfrq(S, T, P)[0]
    n2=n2*10000 #multiply with 10000
    
    #calculate Brunt-Väisälä mannual
    ### N^2=-(g/ρ0)Δρ/ΔP, we assume constant rho0,g
    g=9.81 # units [m s^-2]
    rho0=1025 #units [kg]
    N2=MA.zeros(len(P)-1,'f',fill_value=-999.999)
    for i in range(len(N2)):
        dif_pden=rho[i]-rho[i+1]
        dif_z=P[i]-P[i+1]
        N2[i]=-(g/rho0)* dif_pden / (-dif_z )
    
    #plot CTD stations profiles
    fig=plt.figure(figsize=(15,6))
    ax1, ax2, ax3,ax4 = fig.subplots(1,4,sharey=True)
    plt.suptitle('Station '+label+' (lon: '+lon +', '+'lat: '+lat+', date: '+date+' '+time+')', fontsize=14)
    #plot temperatures
    ax1.plot(T,data['P'],'k')
    ax1.plot(theta, data['P'], 'g')
    ax1.legend(['In-Situ','Potential'],loc=4)
    ax1.set_xlabel('Temperature ($^{\circ}$C)',fontsize=12)
    ax1.xaxis.set_label_position('top') # this moves the label to the top
    ax1.xaxis.set_ticks_position('top') # this moves the ticks to the top
    ax1.set_ylabel('Pressure (dbar)')
    ax1.set_ylim(ax1.get_ylim()[::-1])
    #configure x-axis limits // diferent limits for polar and equatorial stations
    if (abs(float(lat))>abs(60)):
        ax1.set_xlim(-2,9.1)
    else:
        ax1.set_xlim(0,31)
    #plot salinity
    ax2.plot(data['S'],data['P'],'r')
    ax2.set_xlabel('Salinity (psu)',fontsize=12)
    ax2.xaxis.set_label_position('top')
    ax2.xaxis.set_ticks_position('top') 
    ax2.yaxis.set_visible(False) # This erases the y ticks
    #plot density
    ax3.plot(rho_theta,data['P'],'y')
    #ax3.plot(rho_theta, data['P'], 'coral')
    #ax3.legend(['Potential','In-Situ'],loc=4)
    ax3.set_xlabel('Potential Density, $\sigma_0$ (Kg m$^{-3}$)',fontsize=12)
    ax3.xaxis.set_label_position('top')  
    ax3.xaxis.set_ticks_position('top') 
    #ax3.xaxis.set_major_formatter(FormatStrFormatter('%4.f'))
    ax3.yaxis.set_visible(False) 
    #plot Brunt-Väisälä freq
    ax4.plot(n2,data['P'][:-1],'c')
    ax4.set_xlabel('Brunt-Väisälä frequency (s$^{-2}\cdot10^4$)',fontsize=12)
    ax4.xaxis.set_label_position('top') 
    ax4.xaxis.set_ticks_position('top') 
    #ax4.xaxis.set_major_formatter(FormatStrFormatter('%.4f'))
    ax4.yaxis.set_visible(False) 
    
    plt.subplots_adjust(left=0, wspace=0.05, top=0.8)
    
    return fig

def dens_iso_domain(T,S,ax):
    '''This function creates density isolines reference domain in order to make temperature-salinity (TS) diagram
    T,S: list with the min and max of potential temperature, salinity profile respectively.
    ax: figure axes
    '''
    
    # Figure out boudaries (mins and maxs)
    smin = min(S) - (0.01 * min(S))
    smax = max(S) + (0.01 * max(S))
    tmin = min(T) - (0.1 * max(T))
    tmax = max(T) + (0.1 * max(T))
 
    # Calculate how many gridcells we need in the x and y dimensions
    xdim = round((smax-smin)/0.1+1,0)
    ydim = round((tmax-tmin)+1,0)
 
    # Create empty grid of zeros
    dens =np.zeros((int(ydim),int(xdim)))
 
    # Create temp and salt vectors of appropiate dimensions
    ti = np.linspace(1,ydim-1,ydim)+tmin
    si = np.linspace(1,xdim-1,xdim)*0.1+smin
 
    # Loop to fill in grid with densities
    for j in range(0,int(ydim)):
        for i in range(0, int(xdim)):
            dens[j,i]=sw.dens(si[i],ti[j],0)
 
    # Substract 1000 to convert to sigma-t
    dens = dens - 1000
    
    # Plot data 
    CS1=ax.contour(si,ti,dens, linestyles='dashed', colors='k')
    plt.clabel(CS1, fontsize=12, inline=1, fmt='%1.0f') # Label every second level
    #ax1.plot(S,T,'r')
    ax.set_xlabel('Salinity (psu)')
    ax.set_ylabel('Potential temperature ($^{\circ}$C)')
    return 